<a href="https://colab.research.google.com/github/GM-Praveen/Projects/blob/main/langcahin_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.

In [ ]:
!pip install langchain transformers torch sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.6 MB/s eta 0:00:0000:0100:01
  Attempting unins

In [ ]:
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import tempfile

2025-08-06 06:40:07.971666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754462408.379585      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754462408.512187      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def process_pdf(pdf_file):
    global db, qa_chain

    try:
        # Get path from uploaded file (Gradio returns NamedString object with .name)
        file_path = pdf_file.name

        # Load and split document
        loader = PyPDFLoader(file_path)
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(documents)

        # Embeddings + FAISS
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        db = FAISS.from_documents(docs, embeddings)

        # LLM pipeline
        from transformers import pipeline
        hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", max_length=512)
        llm = HuggingFacePipeline(pipeline=hf_pipeline)

        # QA Chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            return_source_documents=False
        )

        return "✅ PDF uploaded and processed successfully! You can now ask questions."

    except Exception as e:
        import traceback
        print("❌ PDF Processing Error:", traceback.format_exc())
        return f"❌ Error while processing PDF:\n{str(e)}"


In [ ]:
# Handle question asking
def ask_question(question):
    if qa_chain is None:
        return "❗ Please upload and process a PDF first."
    if not question.strip():
        return "❗ Please enter a valid question."

    prompt = (
        f"Please answer the following question based on the uploaded document. "
        f"Provide a clear, concise, and easy-to-understand summary in bullet points. "
        f"Focus on the key points only and avoid lengthy explanations.\n\n"
        f"Question: {question}"
    )

    try:
        result = qa_chain({"query": prompt})
        return result["result"]
    except Exception as e:
        return f"⚠️ Error: {e}"

In [ ]:
# Interactive Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        <div style="text-align: center; font-size: 28px; font-weight: bold; color: #2c3e50;">
        📄 Intelligent PDF Q&A Chatbot
        </div>
        <div style="text-align: center; font-size: 16px; color: #7f8c8d;">
        Upload a PDF and ask any question. Get summarized bullet-point answers instantly.
        </div>
        """,
    )

    with gr.Row():
        with gr.Column():
            pdf_file = gr.File(label="📁 Upload PDF", file_types=[".pdf"])
            process_btn = gr.Button("🚀 Process PDF", variant="primary")
            status_box = gr.Textbox(label="📌 Status", interactive=False)

        with gr.Column():
            question_box = gr.Textbox(label="💬 Ask a Question", placeholder="e.g. What is cloud computing?")
            answer_box = gr.Textbox(label="🤖 Bot Answer", lines=10)
            ask_btn = gr.Button("🧠 Get Answer")

    process_btn.click(fn=process_pdf, inputs=pdf_file, outputs=status_box)
    ask_btn.click(fn=ask_question, inputs=question_box, outputs=answer_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7864
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://01d5f7b0736d07ebb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
